# Initial imports

In [1]:
%pip install ipython-autotime  --upgrade
%pip install pandas            --upgrade
%pip install scikit-learn      --upgrade

In [2]:
from google.colab import drive, files
import joblib
import pandas as pd
drive.mount('/content/drive', force_remount=True)
%load_ext autotime

Mounted at /content/drive
time: 448 µs (started: 2023-07-13 23:53:37 +00:00)


In [19]:
dataset = joblib.load("/content/drive/MyDrive/datasets/loterias/loterias-lotofacil.pkl")
#dataset['Data_Sorteio'] = pd.to_datetime(dataset['Data_Sorteio'], format="%d/%m/%Y")
#dataset = dataset.set_index('Data_Sorteio')
dataset = dataset.drop(['Data_Sorteio'], axis=1)

time: 38.7 ms (started: 2023-07-14 00:02:19 +00:00)


# EDA

In [4]:
dataset

,Bola1,Bola2,Bola3,Bola4,Bola5,Bola6,Bola7,Bola8,Bola9,Bola10,Bola11,Bola12,Bola13,Bola14,Bola15
0,2.0,3.0,5.0,6.0,9.0,10.0,11.0,13.0,14.0,16.0,18.0,20.0,23.0,24.0,25.0
1,1.0,4.0,5.0,6.0,7.0,9.0,11.0,12.0,13.0,15.0,16.0,19.0,20.0,23.0,24.0
2,1.0,4.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,14.0,16.0,17.0,20.0,23.0,24.0
3,1.0,2.0,4.0,5.0,8.0,10.0,12.0,13.0,16.0,17.0,18.0,19.0,23.0,24.0,25.0
4,1.0,2.0,4.0,8.0,9.0,11.0,12.0,13.0,15.0,16.0,19.0,20.0,23.0,24.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2706,1.0,2.0,4.0,5.0,8.0,9.0,10.0,11.0,14.0,15.0,16.0,17.0,18.0,21.0,25.0
2707,1.0,2.0,3.0,5.0,6.0,13.0,14.0,16.0,18.0,19.0,21.0,22.0,23.0,24.0,25.0
2708,1.0,2.0,3.0,5.0,9.0,10.0,11.0,12.0,13.0,19.0,21.0,22.0,23.0,24.0,25.0
2709,1.0,2.0,3.0,4.0,7.0,10.0,11.0,12.0,13.0,14.0,16.0,17.0,20.0,24.0,25.0


time: 36 ms (started: 2023-07-13 23:53:37 +00:00)


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2711 entries, 0 to 2710
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Bola1   2711 non-null   float64
 1   Bola2   2711 non-null   float64
 2   Bola3   2711 non-null   float64
 3   Bola4   2711 non-null   float64
 4   Bola5   2711 non-null   float64
 5   Bola6   2711 non-null   float64
 6   Bola7   2711 non-null   float64
 7   Bola8   2711 non-null   float64
 8   Bola9   2711 non-null   float64
 9   Bola10  2711 non-null   float64
 10  Bola11  2711 non-null   float64
 11  Bola12  2711 non-null   float64
 12  Bola13  2711 non-null   float64
 13  Bola14  2711 non-null   float64
 14  Bola15  2711 non-null   float64
dtypes: float64(15)
memory usage: 317.8 KB
time: 15.6 ms (started: 2023-07-13 23:53:37 +00:00)


In [6]:
dataset.describe()

,Bola1,Bola2,Bola3,Bola4,Bola5,Bola6,Bola7,Bola8,Bola9,Bola10,Bola11,Bola12,Bola13,Bola14,Bola15
count,2711.000000,2711.000000,2711.000000,2711.000000,2711.000000,2711.000000,2711.000000,2711.000000,2711.00000,2711.000000,2711.000000,2711.000000,2711.000000,2711.000000,2711.000000
mean,1.628181,3.265585,4.883069,6.524161,8.161933,9.791959,11.388049,13.018812,14.61564,16.257838,17.910365,19.546293,21.174105,22.788639,24.400221
std,0.929867,1.303809,1.541305,1.710632,1.815667,1.860114,1.886422,1.908810,1.92397,1.876124,1.780503,1.649549,1.482969,1.267809,0.943566
min,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.00000,10.000000,12.000000,13.000000,15.000000,17.000000,18.000000
25%,1.000000,2.000000,4.000000,5.000000,7.000000,8.000000,10.000000,12.000000,13.00000,15.000000,17.000000,18.000000,20.000000,22.000000,24.000000
50%,1.000000,3.000000,5.000000,6.000000,8.000000,10.000000,11.000000,13.000000,15.00000,16.000000,18.000000,20.000000,21.000000,23.000000,25.000000
75%,2.000000,4.000000,6.000000,8.000000,9.000000,11.000000,13.000000,14.000000,16.00000,18.000000,19.000000,21.000000,22.000000,24.000000,25.000000
max,7.000000,9.000000,12.000000,13.000000,14.000000,16.000000,17.000000,18.000000,19.00000,20.000000,21.000000,22.000000,23.000000,24.000000,25.000000


time: 59.9 ms (started: 2023-07-13 23:53:37 +00:00)


# Feature engineering

In [20]:
from sklearn.base     import BaseEstimator, TransformerMixin
from sklearn.compose  import ColumnTransformer
from sklearn.pipeline import Pipeline

import itertools
import numpy as np

time: 858 µs (started: 2023-07-14 00:02:23 +00:00)


In [ ]:
cols_names = list(dataset.columns)
cols_names

In [22]:
class RepeatedOne(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self

  def fit_transform(self, X, y=None):
    self.fit(X, y)
    return self.transform(X, y)

  def get_feature_names_out(self):
    pass

  def match_numbers(self, X):
      raffle_x = set(X.iloc[:15])
      raffle_y = set(X.iloc[15:])
      return len(raffle_x.intersection(raffle_y))

  def transform(self, X, y=None):
    X_ = X.copy()
    for name in X_.columns:
      X_['lag1_' + name] = X_[name].shift(1)
    X['repeated'] = X_.apply(self.match_numbers, axis=1)
    return X

ct_repeated = ColumnTransformer(transformers=[('repeated_one', RepeatedOne(), cols_names)], remainder='passthrough')
ct_repeated.set_output(transform='pandas')

pp_repeated = Pipeline(steps=[('repeated_one', RepeatedOne())])
result = pp_repeated.fit_transform(dataset)
result

,Bola1,Bola2,Bola3,Bola4,Bola5,Bola6,Bola7,Bola8,Bola9,Bola10,Bola11,Bola12,Bola13,Bola14,Bola15,repeated
0,2.0,3.0,5.0,6.0,9.0,10.0,11.0,13.0,14.0,16.0,18.0,20.0,23.0,24.0,25.0,0
1,1.0,4.0,5.0,6.0,7.0,9.0,11.0,12.0,13.0,15.0,16.0,19.0,20.0,23.0,24.0,9
2,1.0,4.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,14.0,16.0,17.0,20.0,23.0,24.0,11
3,1.0,2.0,4.0,5.0,8.0,10.0,12.0,13.0,16.0,17.0,18.0,19.0,23.0,24.0,25.0,9
4,1.0,2.0,4.0,8.0,9.0,11.0,12.0,13.0,15.0,16.0,19.0,20.0,23.0,24.0,25.0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2706,1.0,2.0,4.0,5.0,8.0,9.0,10.0,11.0,14.0,15.0,16.0,17.0,18.0,21.0,25.0,10
2707,1.0,2.0,3.0,5.0,6.0,13.0,14.0,16.0,18.0,19.0,21.0,22.0,23.0,24.0,25.0,8
2708,1.0,2.0,3.0,5.0,9.0,10.0,11.0,12.0,13.0,19.0,21.0,22.0,23.0,24.0,25.0,11
2709,1.0,2.0,3.0,4.0,7.0,10.0,11.0,12.0,13.0,14.0,16.0,17.0,20.0,24.0,25.0,9


time: 361 ms (started: 2023-07-14 00:03:46 +00:00)


In [16]:
class EvenOdd(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self

  def fit_transform(self, X, y=None):
    self.fit(X, y)
    return self.transform(X, y)

  def get_feature_names_out(self):
    pass

  def even_amount(self, raffle):
    raffle_size = 15
    even = itertools.filterfalse(lambda x: x % 2 != 0, raffle.iloc[:raffle_size])
    return len(list(even))

  def transform(self, X, y=None):
    X = X.copy()
    raffle_size = 15
    X['even_amount'] = X.apply(self.even_amount, axis=1)
    X['odd_amount']  = raffle_size - X['even_amount']
    return X

ct_evenodd = ColumnTransformer(transformers=[('even_odd', EvenOdd(), cols_names)], remainder='passthrough')
ct_evenodd.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('even_odd', EvenOdd(),
                                 ['Bola1', 'Bola2', 'Bola3', 'Bola4', 'Bola5',
                                  'Bola6', 'Bola7', 'Bola8', 'Bola9', 'Bola10',
                                  'Bola11', 'Bola12', 'Bola13', 'Bola14',
                                  'Bola15'])])

time: 28.6 ms (started: 2023-07-13 23:59:59 +00:00)


In [18]:
class Quartiles(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self

  def fit_transform(self, X, y=None):
    self.fit(X, y)
    return self.transform(X, y)

  def get_feature_names_out(self):
    pass

  def transform(self, X, y=None):
    X_ = X.iloc[:, 0: 15].copy()
    X['quartile_1_5']   = X_.apply(lambda y: ((y >= 1) & (y <= 5)).sum(), axis=1)
    X['quartile_6_10']  = X_.apply(lambda y: ((y >= 6) & (y <= 10)).sum(), axis=1)
    X['quartile_11_15'] = X_.apply(lambda y: ((y >= 11) & (y <= 15)).sum(), axis=1)
    X['quartile_16_20'] = X_.apply(lambda y: ((y >= 16) & (y <= 20)).sum(), axis=1)
    X['quartile_21_25'] = X_.apply(lambda y: ((y >= 21) & (y <=25)).sum(), axis=1)
    return X

ct_quartiles = ColumnTransformer(transformers=[('quartiles', Quartiles(), cols_names)], remainder='passthrough')
ct_quartiles.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('quartiles', Quartiles(),
                                 ['Bola1', 'Bola2', 'Bola3', 'Bola4', 'Bola5',
                                  'Bola6', 'Bola7', 'Bola8', 'Bola9', 'Bola10',
                                  'Bola11', 'Bola12', 'Bola13', 'Bola14',
                                  'Bola15'])])

time: 29.1 ms (started: 2023-07-14 00:00:48 +00:00)


# Data visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
sns.set(rc={'figure.figsize':(15, 5)})
sns.scatterplot(dataset.mean(axis=1))
plt.tight_layout()
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(15, 5)})
sns.scatterplot(dataset.std(axis=1))
plt.tight_layout()
plt.show()

In [ ]:
fig, axis = plt.subplots(2, 1, figsize=(20, 7))
axis = axis.ravel()
sns.lineplot(dataset['even_amount'], ax=axis[0])
sns.lineplot(dataset['odd_amount'], ax=axis[1])
plt.tight_layout()
plt.show()